In [1]:
# Gerekli kütüphaneler
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Cihaz Kontrolü (GPU Kullanımı)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Hiperparametreler
num_epochs = 10
batch_size = 100
learning_rate = 0.02

In [4]:
# MNIST Veri Kümesini Yükleme ve Normalizasyon
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# ANN Modeli Tanımlama
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        
        # Katmanlar
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.tanh2 = nn.Tanh()
        
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.elu3 = nn.ELU()
        
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        
        out = self.fc2(out)
        out = self.tanh2(out)
        
        out = self.fc3(out)
        out = self.elu3(out)
        
        out = self.fc4(out)
        return out

In [7]:
# Modeli Tanımla ve Cihaza Gönder
input_dim = 28 * 28
hidden_dim = 150
output_dim = 10
model = ANNModel(input_dim, hidden_dim, output_dim).to(device)

# Kayıp Fonksiyonu ve Optimizasyon
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [8]:
# Model Eğitimi
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Görselleri düzleştir ve GPU/CPU'ya gönder
        images = images.view(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # Gradients sıfırlama
        optimizer.zero_grad()

        # İleri Yayılım
        outputs = model(images)

        # Kayıp Hesaplama
        loss = criterion(outputs, labels)

        # Geri Yayılım
        loss.backward()

        # Parametre Güncelleme
        optimizer.step()

        count += 1

        # Her 50 iterasyonda bir doğruluk ölç
        if count % 50 == 0:
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in test_loader:
                    images = images.view(-1, 28 * 28).to(device)
                    labels = labels.to(device)

                    outputs = model(images)
                    predicted = torch.argmax(outputs, dim=1)

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            accuracy = 100 * correct / total
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if count % 500 == 0:
                print(f'Iteration: {count}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

Iteration: 500, Loss: 0.5687, Accuracy: 86.58%
Iteration: 1000, Loss: 0.4353, Accuracy: 90.22%
Iteration: 1500, Loss: 0.2341, Accuracy: 91.83%
Iteration: 2000, Loss: 0.2817, Accuracy: 92.56%
Iteration: 2500, Loss: 0.1452, Accuracy: 93.12%
Iteration: 3000, Loss: 0.2091, Accuracy: 94.35%
Iteration: 3500, Loss: 0.3292, Accuracy: 94.87%
Iteration: 4000, Loss: 0.1078, Accuracy: 95.34%
Iteration: 4500, Loss: 0.2218, Accuracy: 95.82%
Iteration: 5000, Loss: 0.2656, Accuracy: 94.98%
Iteration: 5500, Loss: 0.2094, Accuracy: 95.13%
Iteration: 6000, Loss: 0.0840, Accuracy: 96.58%
